## W4A16 Quantization and Compression ##

Using compressed-tensors, we can compress a quantized model to store it more efficiently on disk.

In this example, we run post-training quantization (PTQ) to quantize the weights of an example model to 4 bits. We then save a compressed version of the model on disk by packing each group of eight 4-bit weights into a single int32

By packing groups of eight 4-bit weights into a single int32, we can store a quantized model more efficiently on disk.

In [2]:
import torch
import os
from tqdm import tqdm
from compressed_tensors.quantization import (
    QuantizationConfig,
    QuantizationStatus,
    apply_quantization_config,
    compress_quantized_weights
)
from compressed_tensors.compressors import ModelCompressor
from transformers import AutoModelForCausalLM, AutoTokenizer, DefaultDataCollator
from datasets import load_dataset
from torch.utils.data import RandomSampler
from torch.utils.data import DataLoader

In [3]:
# load a dense, unquantized tiny llama model
device = "cuda:0"
model_name = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"
model = AutoModelForCausalLM.from_pretrained(model_name, device_map=device, torch_dtype="auto")
model

config.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.40G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_emb): 

The following quantization config will be used to quantize all of the Linear layers to 4 bits, excluding the lm_head layer. 

The `format` argument is set to `pack-quantized`, indicating that when the model is saved we should use the `PackedQuantizationCompressor` which will pack every eight 4-bit weights into an `int32`. 

This will give us a compression ratio of 8x on each Linear layer compared to the unquantized `float32` representation

In [23]:
quantization_config_dict = {
	"quant_method": "sparseml",
	"format": "pack-quantized",
	"global_compression_ratio": None,
	"config_groups": {
        "group_1": {
            "weights": {
                "num_bits": 4,
                "type": "int",
                "symmetric": False,
                "strategy": "tensor"
            },
            "targets": ["Linear"]
        }
    },
	"ignore": ["lm_head"]
}

In [ ]:
# setup the loaded model for quantization calibration

config = QuantizationConfig(**quantization_config_dict)
config.quantization_status = QuantizationStatus.CALIBRATION
apply_quantization_config(model, config)

In [ ]:
# create a dataloader of calibration data

dataset = load_dataset("ptb_text_only", trust_remote_code=True)["train"]
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding=False, truncation=True, max_length=1024)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

data_loader = DataLoader(
    tokenized_dataset, batch_size=1, collate_fn=DefaultDataCollator(), sampler=RandomSampler(tokenized_dataset)
)

In [28]:
# calibrate scale and zero points for quantization using a small amount of train data
num_calibration_samples = 512

with torch.no_grad():
    for idx, sample in tqdm(enumerate(data_loader), desc="Running calibration"):
        sample = {key: value.to(device) for key,value in sample.items()}
        _ = model(**sample)

        if idx >= num_calibration_samples:
            break

# freeze scale and zero points after calibration
# model.apply(freeze_module_quantization)

Running calibration: 512it [00:33, 15.42it/s]


After running calibration, each quantized layer will have a new scale and zero_point parameter as shown below.

Notice that at this point, the weight itself is still a floating point and has not been quantized. 

To convert the weights to an integer type, we need to apply the `compress_quantized_weights` function. After compressing the weights, a forward pass of the model can no longer be run in PyTorch

In [29]:
state_dict = model.state_dict()
example_layer = "model.layers.0.self_attn.q_proj.weight"
scale = state_dict[example_layer + "_scale"]
zero_point = state_dict[example_layer + "_zero_point"]
weight = state_dict[example_layer]
print(f"Scale: {scale}, Zero Point: {zero_point}")
print(f"Weight min: {torch.min(weight)} max: {torch.max(weight)} dtype: {weight.dtype}")

Scale: tensor([17296.], device='cuda:4', dtype=torch.float16), Zero Point: tensor([0], device='cuda:4', dtype=torch.int8)
Weight min: -1.587890625 max: 1.0283203125 dtype: torch.float16


In [30]:
# convert quantized weights to integers
model.apply(compress_quantized_weights)

state_dict = model.state_dict()
example_layer = "model.layers.0.self_attn.q_proj.weight"
scale = state_dict[example_layer + "_scale"]
zero_point = state_dict[example_layer + "_zero_point"]
weight = state_dict[example_layer]
print(f"Scale: {scale}, Zero Point: {zero_point}")
print(f"Weight min: {torch.min(weight)} max: {torch.max(weight)} dtype: {weight.dtype}")

Scale: tensor([17296.], device='cuda:4', dtype=torch.float16), Zero Point: tensor([0], device='cuda:4', dtype=torch.int8)
Weight min: 0 max: 0 dtype: torch.int8


After compressing the quantized model, the weight matrix has a range of int4 but is stored in an int8. 

We can further compress the model on disk using the `pack-quantized` format we specified in the config. This compression format will pack the int4 weights into int32

In [31]:
# apply compression and save the model to disk

output_dir = "./ex_llama1.1b_w4a16_packed_quantize"
compression_format = config.format
print(f"Compression format: {compression_format}")

compressor = ModelCompressor(quantization_config=config)
compressed_state_dict = compressor.compress(model)
model.save_pretrained(output_dir, state_dict=compressed_state_dict)
compressor.update_config(output_dir)

compressed_size_on_disk_mb = os.path.getsize(os.path.join(output_dir, "model.safetensors")) / 1024 / 1024
print(f"Size of the model's weights on disk using safetensors: {compressed_size_on_disk_mb:.2f} MB")

Compression format: pack-quantized


Quantized Compression: 100%|██████████| 509/509 [00:03<00:00, 153.70it/s]


Size of the model's weights on disk using safetensors: 712.23 MB
